# Caso Financiero - Banco de Colombia

## Business Understanding 

**¿Qué es una tarjeta de crédito?**

Una tarjeta de crédito es un instrumento usado por personas naturales y jurídicas que les permiten acceder rápidamente al crédito.
 
**¿Cómo funcionan?**

Los bancos otorgan las tarjetas a personas asignando un cupo limite que pueden usar, este cupo basado en diversas características de cada persona. 
 
**Aspectos importantes para conocer en una tarjeta de crédito:**

-	Fecha de Corte: Fecha de facturación en un periodo 
-	Fecha límite de pago: Fecha máxima para pagar el monto facturado en la fecha de corte sin generar intereses
-	Intereses corrientes: Intereses generados desde la compra hasta el momento del pago
-	Intereses de mora: Intereses generados después de la fecha límite de pago 
 
**¿Cómo ganan los bancos?**

-   Cuotas de manejo: Una cuota mensual que el propietario de la tarjeta le paga al banco por el uso de la tarjeta.
-   Intereses corrientes: algunas tarjetas generan intereses desde el momento de la compra hasta el momento en el que el usuario realiza el pago.
-	Intereses de mora: Intereses que se generan después de la fecha límite de pago 
-	Comisiones de comercio: Los comercios deberán pagar un porcentaje de la venta al momento de hacer una venta con tarjeta 
 
**¿Por qué las personas usan tarjetas de Crédito?:**

-	Mecanismo eficiente de apalancamiento, permite a las personas usar crédito de manera rápida 
-	Beneficios, muchas tarjetas de crédito otorgan diferentes tipos de beneficios; Descuentos, Puntos y Afiliaciones 
-	Otorga un perfil crediticio a las personas 
 
 
 
**¿Cuál es el proceso en una compra?** 

1.	Lectura de tarjeta: Se lee el chip de la tarjeta, o bajo tecnología NFC se obtienen los datos de la tarjeta
2.	Se ingresa el monto: El vendedor asigna el monto de la compra
3.	Solicitud de autorización: 
-	El datafono de conecta a una red de procesamiento para la autorización 
-	La solicitud se envía al banco emisor de la tarjeta a través de una red de franquicias
-	El banco emisor, verifica la validez, si existen los fondos suficientes disponibles

4.	Autorización o denegación: Si la transacción es aprobada, se envía una autorización al datafono, si es denegada se envía un código de error
5.	Finalización de la transacción: se imprime un recibo de la transacción 
6.	Liquidación: Generalmente al finalizar la jornada el banco le paga al comercio el monto total del día, sacando las comisiones asociadas a la operación.

**¿Qué son y cómo funcionan las franquicias?**

- Las franquicias son una red de procesamiento de pagos, aunque no emiten tarjetas directamente a los consumidores en su lugar proporcionan la infraestructura para que se emitan las tarjetas y procesen transacciones
 
**¿Como ganan estás franquicias?**

-	Comisiones de transacción
-	Tarifas de licencia a bancos e instituciones financieras 
-	Tarifas de servicio de valor agregado 
-	Ingresos por inversión 
-	Partnerschips y colaboraciones 
 

**Principales bancos Colombianos (participación de mercado)** 

- 	Bancolombia 26.5%
- 	Davivienda 16.5%
- 	Banco de Bogotá 11.8%
- 	BBVA 10.6%
- 	Banco de Occidente 6.2%

**Objetivo del negocio**

-	Aumentar el volumen de tarjetas de crédito en el mercado

**Criterio de éxito**

Al aumentar el volumen de tarjetas de crédito es aún más rentable si se consiguen personas con las siguientes características:
-	Historial crediticio sólido: Un cliente de pagar sus deudas a tiempo es atractivo
-	Estabilidad financiera: Una situación laboral estable, ingresos regulares y relación saludable entre ingresos / gastos 
-	Uso regular de la tarjeta: Uso regular de la tarjeta, genera comisiones al banco 
-	Mantener un saldo, pero pagar el mínimo: es rentable una persona que además de usar la tarjeta de crédito regularmente, mantenga un saldo y pague el mínimo, le hace ganar al banco por el concepto de comisiones de mora, el banco gana por comisiones de compra y además reduce un poco el riesgo al pagar el valor mínimo de pago 
-	No supera su límite de crédito: es amigable una persona que maneje su riesgo de crédito y no supere su límite de crédito 
-	Historial de trabajo y residencia estable 
-	Bajo nivel de endeudamiento 
-	Años de historial crediticio 
-	Diversidad en tipos de crédito 

**Objetivo de la minería de datos:**
-	Poder identificar los diferentes tipos de clientes que posee la empresa, para hacer promociones eficientemente

**Criterio de éxito de la minería de datos**
-	Identificación clara de los grupos de clientes
-	Identificación de hábitos de consumo 
-	Identificación de posibles usuarios riesgosos (personas que por sus hábitos puede ser mejor no ofrecerles promociones)



## Data understanding


### Data description report

En la exploración de los datos nos encontramos con una base de datos que contiene registros de 47871 clientes de tarjetas de un banco que maneja tres franquicias: visa, mastercard y una independiente del banco. Las variables que nos encontramos en cada uno de estos registros nos indican pertenencia a un grupo (del cual no tenemos información), estadística sobre las compras del usuario, la franquicia que usa, uso nacional o internacional, hora del día, día de la semana y tipo de comercio donde más compras realizó, totalizando 25 variables. 

El formato predominante de los datos es el formato de porcentaje, pues es así como está determinado el uso de franquicias sobre el uso total de todas, el uso nacional o internacional con las franquicias y en el total, el uso en cada rango horario y el uso en cada día de la semana; por otro lado, los datos como el mínimo, máximo y promedio de compra se encuentra en términos nominales de dinero, presumiblemente pesos colombianos y el tipo de comercio preferido como una variable categórica.

### Data exploration

Para la exploración de datos decidimos separar los registros en función del sitio preferido de los clientes, pues graficar en función de cada registro resulta inviable por la cantidad que tenemos y además carente de significado para el análisis, en cambio, los sitios preferidos si nos dan una visión significativa para las demás variables que tenemos. Por lo tanto será así como exploremos los datos y los trabajemos más adelante, como forma de exploración hicimos previamente un gráfico de barras donde se representa el valor total de compras en cada comercio usando visas en territorio nacional:

<img src='Imagen4.png'>

Aquí vemos entonces que con visas en territorio nacional los clientes del banco usan mayormente sus tarjetas para comprar en SUPERMERCADOS Y TIENDAS EXPRESS, ALMACEN POR DEPARTAMENTO CON SUPERMERCADO, ALMACENES DE VESTUARIO Y ACCESORIO PARA LA FAMILIA, ALAMCENES DE CALZADO, etc.

Este modo de ver la información nos resulta particularmente útil para poder sugerir al banco que tipo de promociones lanzar, pues podremos determinar que combinación de variables resultará más efectiva por la cantidad de clientes interesados en aprovecharlas gracias a esta observación y análisis de su comportamiento.

### Data quality

No encontramos errores para corregir en la base de datos en sí, sin embargo hay varibles que no serán tomadas en cuenta para el desarrollo del análisis de los datos por su carencia de significado para el mismo, tal como el grupo del cliente, el mínimo, máximo y desviación estándar de las compras, porcentaje nacional e internacional total, debido a que ya tenemos la misma información con las franquicias y de los sitios de consumo excluiremos "SIN NOMBRE".

## Data preparation

Primero, importamos la lobrerías que nos serán útiles para el desarrollo del proyecto:

In [1]:
import pandas as pd
import hvplot.pandas
from path import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

Importamos los datos contenidos en el documento de excel infobanca.xlsx

In [2]:
infobanca= pd.read_excel('infobanca.xlsx')
#infobanca= pd.read_excel("C:\\Users\\migue\\Documents\\GitHub\\Fintech_and_Machine_Learning\\Caso1\\infobanca.xlsx")
infobanca_original=infobanca

### Data construction

Ahora creamos columna de total de ventas multiplicando el promedio por transacción por el número de transacciones con el fin de tener un número total aproximado del consumo de cada uno de los usuarios:

In [3]:
infobanca['Total_ventas']=infobanca['promedio_por_transaccion']*infobanca['Numero_de_transacciones']

### Data cleaning

Eliminamos las columnas mencionadas previamente en el reporte de calidad de los datos y además eliminamos la columna del promedio por transacción, pues al tener el total consumido por cada usuario, no lo necesitamos de ahora en adelante. Eliminamos también los registros de usuarios cuyo sitio preferido haya sido "SIN NOMBRE":

In [4]:
coldt=['CLIENTE','grupo_de_cliente','Numero_de_transacciones','promedio_por_transaccion','transaccion_minima','transaccion_maxima','desviacion_estandar_por_transaccion','porcentaje_nacional_total','porcentaje_internacional_total']
infobanca= infobanca.drop(coldt, axis=1)
infobanca= infobanca[infobanca['Sitio_consumo_masfrecuente'] !='SIN NOMBRE']
infobanca.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO,Sitio_consumo_masfrecuente,Total_ventas
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,CLINICAS - HOSPITALES,459930.3
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,0.0,0.0,0.333333,0.0,0.0,MERCADEO DIRECTO - COMERCIANTES DE VENTAS TELE...,1748000.1
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,"DROGUERIAS, FARMACIAS, TIENDAS NATURISTAS",1849641.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.4,0.6,0.0,0.0,0.600000,0.2,0.0,0.100000,0.0,0.1,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,1444671.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,321764.0


### Data formatting

Multiplicamos total venta por los porcentajes del DF con el fin de deshacernos de los porcentajes y en vez de eso tener valores netos en pesos colombianos de las ventas hechas con cada franquicia, hora del día, etc.

In [5]:
colum = list(infobanca.columns)
colum.remove('Sitio_consumo_masfrecuente')
colum.remove('Total_ventas')
for i in colum:
  infobanca[i] = infobanca[i]*infobanca['Total_ventas']
infobanca.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO,Sitio_consumo_masfrecuente,Total_ventas
0,0.0,459930.3,0.0,0.0,0.0,0.0,0.0,0.0,459930.3,0.0,0.0,459930.3,0.0,0.0,0.0,0.0,CLINICAS - HOSPITALES,459930.3
1,1748000.1,0.0,0.0,0.0,0.0,0.0,0.0,1748000.1,0.0,0.0,1165333.4,0.0,0.0,582666.7,0.0,0.0,MERCADEO DIRECTO - COMERCIANTES DE VENTAS TELE...,1748000.1
2,1849641.0,0.0,0.0,0.0,0.0,0.0,1849641.0,0.0,0.0,0.0,0.0,0.0,1849641.0,0.0,0.0,0.0,"DROGUERIAS, FARMACIAS, TIENDAS NATURISTAS",1849641.0
3,0.0,0.0,1444671.0,0.0,0.0,0.0,577868.4,866802.6,0.0,0.0,866802.6,288934.2,0.0,144467.1,0.0,144467.1,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,1444671.0
4,321764.0,0.0,0.0,0.0,0.0,0.0,0.0,321764.0,0.0,321764.0,0.0,0.0,0.0,0.0,0.0,0.0,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,321764.0


Ahora cambiamos la forma de la base de datos, pues en vez de tener un cliente en cada fila, tendremos cada uno de los sitios y las variables que quedarán registradas serán las sumatorias del total de dinero invertido en cada sitio bajo cada criterio, tal como las compras con visa nacional o internacional, la hora del día de la transacción, etc.

Esta será la base de datos con las que iniciemos la preparación de los datos para la aplicación del metodo de clustering K-means.

In [6]:
columnas=list(infobanca.columns.values)
columnas.remove('Total_ventas')
suma_por_sitio=infobanca.groupby('Sitio_consumo_masfrecuente')[columnas].sum()
infobanca=suma_por_sitio

infobanca.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO
Sitio_consumo_masfrecuente,,,,,,,,,,,,,,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",1.030147e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,1.030147e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.801470e+05,0.000000e+00,0.000000e+00,0.000000e+00,4.500000e+05
AEROLINEAS,2.186256e+09,7.111309e+08,1.435605e+09,4.533029e+08,31649905.85,97490739.12,1.491057e+09,2.369273e+09,1.055105e+09,4.472980e+08,6.592852e+08,7.800953e+08,8.038844e+08,7.799944e+08,8.536865e+08,5.911916e+08
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,4.731503e+07,5.798083e+06,2.098882e+07,2.363915e+06,161590.00,442522.84,2.166422e+07,3.781246e+07,1.759329e+07,1.168837e+07,5.219402e+06,1.181228e+07,1.345278e+07,1.116347e+07,7.654751e+06,1.607891e+07
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,6.206000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,6.206000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.206000e+05,0.000000e+00
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,1.400711e+09,1.114064e+08,5.258823e+08,5.848946e+07,8172345.31,32829322.66,6.804893e+08,1.265260e+09,1.917410e+08,1.247545e+08,3.298423e+08,3.639641e+08,3.316032e+08,3.434155e+08,3.637598e+08,2.801513e+08


Para un mejor análisis y lectura de los datos vamos a simplificar la información dejándola en **MILLONES**

In [7]:
colum= infobanca.columns
for i in colum:
    infobanca[i]=infobanca[i]/1000000

In [8]:
infobanca=round(infobanca,2)

### Escalar datos

Escalamos la base de datos con standard sca `StandardScaler` para normalizar los datos y creamos el índice `Sitio_consumo_masfrecuente`

In [9]:
infobanca_escalado=StandardScaler().fit_transform(infobanca)

# DataFrame con los datos escalados
df_infobanca_scaled = pd.DataFrame(
    infobanca_escalado,
    columns=infobanca.columns
)

# Copiamos los nombres de los sitios
df_infobanca_scaled["Sitio_consumo_masfrecuente"] = infobanca.index

# Establecer la columna Sitio_consumo_masfrecuente como índice
df_infobanca_scaled= df_infobanca_scaled.set_index("Sitio_consumo_masfrecuente")

df_infobanca_scaled.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO
Sitio_consumo_masfrecuente,,,,,,,,,,,,,,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",-0.405491,-0.288622,-0.356500,-0.295740,-0.334358,-0.159009,-0.413602,-0.397871,-0.362379,-0.318788,-0.406218,-0.426617,-0.425587,-0.428480,-0.424400,-0.364972
AEROLINEAS,4.001300,3.815425,1.414048,3.221043,0.978403,1.108289,3.111974,2.487574,3.576975,1.826478,2.898515,3.336319,3.377035,3.281451,3.558383,1.987535
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,-0.312142,-0.255150,-0.330612,-0.277430,-0.327722,-0.153289,-0.364789,-0.351824,-0.296705,-0.262722,-0.380052,-0.372407,-0.361964,-0.375398,-0.388710,-0.302729
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,-0.406318,-0.288622,-0.356500,-0.295740,-0.334358,-0.159009,-0.414572,-0.397871,-0.362379,-0.318788,-0.406218,-0.429417,-0.425587,-0.428480,-0.421507,-0.366764
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,2.417140,0.354343,0.292072,0.158036,0.004513,0.267757,1.194076,1.143041,0.353501,0.279517,1.247126,1.327508,1.142993,1.204957,1.272676,0.748879


# Modelling

## Select Modelling Techniques

Recordando nuestro objeto de estudio, buscamos a través de grupos poblacionales agrupados por sitio donde más frecuentan sus compras, presentar posibles promociones que benefician a nuestros clientes, a través del medio de pago que emplean en dichos sitios. 

Ahora bien, dado que necesitamos sugerir las promociones para los individuos, estas estarán representadas por sitios, recordando que la base de datos fue ajustada de esa manera anteriormente. Teniendo en cuenta que las promociones se harán de acuerdo con las variables que tenemos como tipos de tarjetas, franjas horarias o dias de la semana, la metodología que vamos a emplear será K-Means, en donde el algoritmo nos permitirá identificar los puntos que más tengan similitud. 

Sabiendo que esta metodología necesita un K optimo donde minimiza la varianza entre puntos y la maximiza entre centroides, agrupandolos de acuerdo a dicho K, partimos de los datos originales para hallar este K, primero tomamos números del 1 al 15, con los que trazaremos los valores de inercia y poder hallar el k óptimo para los clusters.


## Build Model

El modelo que vamos a utilizar será K-Means, ahora bien, como lo mencionamos anteriormente, este algoritmo tiene dos parámetros importantes, el primero el K que vamos a emplear, el cual será seleccionado bajo el método de codo, el cual utiliza la distancia media de las observaciones a su centroide. En otras palabras, busca fijar las distancias intra-cluster, en donde a medida que el K es mayor la varianza tiende a disminuir, lo que significaría que los clústeres son más compactos.

### Método de codo

In [10]:
k = list(range(1,15))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [11]:
# Crear una lista vacía para almacenar los valores de inertia
inertia = []

In [12]:
# Bucle for para calcular la inercia de cada K
for i in k:
    model = KMeans(n_clusters = i, random_state = 0)
    model.fit(df_infobanca_scaled)
    inertia.append(model.inertia_)

inertia

[1728.0,
 695.8366186458875,
 431.4127361062938,
 297.4403716663284,
 198.1877527289729,
 127.10795536254786,
 90.23584995838549,
 58.09242712561695,
 42.11419486615875,
 33.81132904640282,
 27.52049564064222,
 21.8201937136159,
 17.55488274075637,
 13.832177266768815]

In [13]:
# Crear un diccionario con los datos para trazar la curva del Codo
elbow_data_original = {
    'k': k,
    'inertia': inertia
}

# Crea un DataFrame con los datos para trazar la curva del Codo
df_elbow_data_original = pd.DataFrame(elbow_data_original)
df_elbow_data_original

,k,inertia
0,1,1728.000000
1,2,695.836619
2,3,431.412736
3,4,297.440372
4,5,198.187753
5,6,127.107955
6,7,90.235850
7,8,58.092427
8,9,42.114195
9,10,33.811329


In [14]:
# Observamos los valores de inercia en un gráfico de línea para encontrar el K óptimo

elbow_plot_original = df_elbow_data_original.hvplot.line(
    title = 'Elbow Plot: Original Data',
    x = 'k',
    xlabel = 'k',
    xticks = k,
    y = 'inertia',
    ylabel = 'Inertia'
)

elbow_plot_original

A partir del gráfico, el mejor valor de K parece ser el K=6, el cual será el parametro a utilizar como número de clusteres que se formaran.

### Clusters ~ K-Means

Ahora armaremos clusters de los sitios de consumo más frecuentes usando el K=6 que encontramos en el paso anterior.

In [15]:
# Inicializar el modelo K-Means usando el mejor valor para k
model = KMeans(n_clusters=6)
model.fit(df_infobanca_scaled) # Ajuste del modelo

In [16]:
# Prediga los grupos para agrupar los sitios de consumo usando los datos escalados
clusters_original = model.predict(df_infobanca_scaled)

# Ver la matriz resultante de valores de clúster.
clusters_original

In [17]:
# Crear una copia del DataFrame
df_clusters_original = infobanca.copy()

Una vez tenemos los Clusteres que vamos a asignara a las observaciones (sitios), vamos a pegar ese vector columna con dicha información con nuestra base de datos original.

In [18]:
# Agregue una nueva columna al DataFrame con los grupos predichos
df_clusters_original['Cluster'] = clusters_original

# Mostrar datos de muestra
df_clusters_original['Cluster']

In [19]:
cluster_plot_original = df_clusters_original.hvplot.scatter(
    title='Cluster Plot: Original',
    x = 'porcentaje_manana',
    xlabel = 'porcentaje_manana',
    y = 'porcSABADO',
    ylabel = 'porcSABADO',
    by = 'Cluster',
    hover_cols=["Sitio_consumo_masfrecuente"]
)

cluster_plot_original

### Optimizar clústeres con análisis de componentes principales

Otra metodología que se puede utilizar con el fin de contrastar los resultados del modelamiento anterior es la optimización de los clústeres, pero bajo un análisis de componentes principales, antes de correr el modelo. Este método lo que nos permitirá será reducir las variables que tengamos a un número definido a discreción del investigador, en este caso dado que tenemos 3 categorías en las que podemos agrupar toda la información (Tipo de tarjeta por franquicias, franja horaria, Noche o Día y dias de la semana) el número de componentes principales será 3.

In [20]:
pca_model = PCA(n_components=3) #Creación del algoritmo
pca_data = pca_model.fit_transform(df_infobanca_scaled) # Reducción a tres CP
pca_data[:5]

Dado que ya tenemos los datos reducidos a tres componentes principales vamos a crear un DataFrame que los contenga. 

In [21]:
df_pca_data = pd.DataFrame(pca_data)
df_pca_data["Sitio_consumo_masfrecuente"] = infobanca.index
df_pca_data = df_pca_data.set_index('Sitio_consumo_masfrecuente')

# Mostrar datos de muestra
df_pca_data.head(5)

Una vez tenemos los datos bajo la metodologia ACP (Analisis de componentes principales), vamos a utilizar estos datos para calcular el K óptimo que se requiere a la ahora de correr nuestro modelo K-Means

In [22]:
k

In [23]:
inertia2 = []
for i in k:
    kmeans_model_pca = KMeans(n_clusters=i, random_state=1)
    kmeans_model_pca.fit(df_pca_data)
    inertia2.append(kmeans_model_pca.inertia_)

inertia2

In [24]:
elbow_data_pca = {
    'k': k,
    'inertia':inertia2
}
df_elbow_data_pca = pd.DataFrame(elbow_data_pca)
df_elbow_data_pca

In [25]:
elbow_plot_pca = df_elbow_data_pca.hvplot.line(
    title='Elbow Plot: PCA',
    x = 'k',
    xticks = k,
    xlabel = 'k',
    y = 'inertia',
    ylabel = 'inertia'
)

elbow_plot_pca

Teniendo en cuenta la gráfica de codo utilizando los datos calculados por componentes principales, notamos que tiene una similitud a la gráfica con los datos originales, por lo que seguiremos utilizando el mismo K, que en este caso será K = 6. 

In [26]:
kmeans_model_pca = KMeans(n_clusters=6)
kmeans_model_pca.fit(df_pca_data)

In [27]:
clusters_pca = kmeans_model_pca.predict(df_pca_data)
clusters_pca

In [28]:
df_clusters_pca = infobanca.copy() #DataFrame PCA
df_clusters_pca['Cluster'] = clusters_pca
df_clusters_pca.head(5)

In [29]:
cluster_plot_original2 = df_clusters_pca.hvplot.scatter(
    title='Cluster Plot: Original',
    x = 'porcentaje_mastercard_nacional',
    xlabel = 'porcentaje_mastercard_nacional',
    y = 'porcDOMINGO',
    ylabel = 'porcDOMINGO',
    by = 'Cluster',
    hover_cols=["Sitio_consumo_masfrecuente"]
)

cluster_plot_original2

In [30]:
cluster_plot_original2 = df_clusters_pca.hvplot.scatter(
    title='Cluster Plot: Original',
    x = 'porcentaje_manana',
    xlabel = 'porcentaje_manana',
    y = 'porcSABADO',
    ylabel = 'porcSABADO',
    by = 'Cluster',
    hover_cols=["Sitio_consumo_masfrecuente"]
)

cluster_plot_original2

# Evaluación 